In [1]:
from PIL import Image
import base64
import requests
from dotenv import load_dotenv
import os
from collections import defaultdict
import os
import random
from tqdm import tqdm

random.seed(42)
load_dotenv()


def resize_image(image_path, size):
    """resize image so that the largest edge is atmost size"""
    img = Image.open(image_path)
    width, height = img.size

    if width <= size and height <= size:
        return img

    if width > height:
        new_width = size
        new_height = int(height * (size / width))
    else:
        new_height = size
        new_width = int(width * (size / height))
    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return img


def encode_image(image_path):
    img = resize_image(image_path, 512)
    temp_name = "temp.jpg"
    img.save(temp_name)
    with open(temp_name, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def construct_mcq(options, correct_option):
    correct_option_letter = None
    i = "a"
    mcq = ""

    for option in options:
        if option == correct_option:
            correct_option_letter = i
        mcq += f"{i}. {option}\n"
        i = chr(ord(i) + 1)

    mcq = mcq[:-1]
    return mcq, correct_option_letter


def add_row(content, data, i, with_answer=False):
    encoded_image = encode_image(data["image_path"])

    content.append(
        {
            "type": "text",
            "text": "Image " + str(i) + ": " + data["question"] + "\n" + data["mcq"],
        }
    )
    content.append(
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}",
            },
        }
    )

    if with_answer:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i) + data["correct_option_letter"],
            }
        )
    else:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i),
            }
        )

    return content

In [2]:
FEWSHOT_JSON = "illusionVQA/sofloc/fewshot_labels.json"
FEWSHOT_IMAGE_DIR = "illusionVQA/sofloc/FEW_SHOTS/"
EVAL_JSON = "illusionVQA/sofloc/eval_labels.json"
EVAL_IMAGE_DIR = "illusionVQA/sofloc/EVAL/"

NIM_URL = "https://ai.api.nvidia.com/v1/gr/meta/llama-3.2-90b-vision-instruct/chat/completions"
MODEL_NAME = "meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo"

In [3]:
import json

with open(FEWSHOT_JSON) as f:
    fewshot_dataset = json.load(f)


for data in fewshot_dataset:
    data["image_path"] = FEWSHOT_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )

In [4]:
with open(EVAL_JSON) as f:
    eval_dataset = json.load(f)

random.shuffle(eval_dataset)

In [ ]:
category_count = defaultdict(int)

for data in eval_dataset:
    if data["image"] not in os.listdir(EVAL_IMAGE_DIR):
        print(data["image"])
        continue
    data["image_path"] = EVAL_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )
    category_count[data["category"]] += 1

print(category_count)
print(len(eval_dataset))

## Trial

In [6]:
content = []

content.append(
    {
        "type": "text",
        "text": "Image 1\n",
    }
)
content.append(
    {
        "type": "image_url",
        "image_url": {
            "url": f"https://gratisography.com/wp-content/uploads/2024/03/gratisography-funflower-1170x780.jpg",
        },
    }
)
content.append(
    {
        "type": "text",
        "text": "Explain the image above",
    }
)

In [ ]:
content

In [8]:
headers = {
    "Authorization": f"Bearer {os.getenv('NIM_API_KEY')}",
    "Accept": "application/json",
}
payload = {
    "model": "meta/llama-3.2-90b-vision-instruct",
    "messages": [{"role": "user", "content": content}],
    "max_tokens": 512,
    "temperature": 1.00,
    "top_p": 1.00,
    "stream": False,
}

In [9]:
response = requests.post(NIM_URL, headers=headers, json=payload)

In [ ]:
# print(response.choices[0].message.content)
print(response.json()["choices"][0]["message"]["content"])

### 0-shot

In [11]:
content = [
    {
        "type": "text",
        "text": "You'll be given an image, an instruction and some options. You have to select the correct one. Do not explain your reasoning. Answer with only the letter which corresponds to the correct option. Do not repeat the entire answer. Do not output anything other than the correct letter. For example, if the correct option is 'a', you should only output 'a'.",
    }
]
next_idx = 1

### 4-shot

In [12]:
# content = [
#     {
#         "type": "text",
#         "text": "You'll be given an image, an instruction and some choices. You have to select the correct one. Do not explain your reasoning. Answer with the option's letter from the given choices directly. Here are a few examples:",
#     }
# ]


# i = 1
# for data in fewshot_dataset:
#     content = add_row(content, data, i, with_answer=True)
#     i += 1

# content.append({
#                     "type": "text",
#                     "text": "Now you try it!",
#                 })

# next_idx = i

### Evaluation Loop

In [13]:
def parse_output(output: str):
    if output.startswith("Answer"):
        return output.split(": ")[1][0].lower()
    else:
        return output[0].lower()

In [14]:
ytrue = []
ypred = []

In [ ]:
import time

MAX_RETRIES = 2
for data in tqdm(eval_dataset):
    content_t = add_row(content.copy(), data, next_idx, with_answer=False)
    payload = {
        "model": "meta/llama-3.2-90b-vision-instruct",
        "messages": [{"role": "user", "content": content}],
        "max_tokens": 512,
        "temperature": 1.00,
        "top_p": 1.00,
        "stream": False,
    }
    payload["messages"][0]["content"] = content_t
    retries = MAX_RETRIES
    while True:
        try:
            response = requests.post(NIM_URL, headers=headers, json=payload)
            response = response.json()["choices"][0]["message"]["content"].strip()
            intern_ans = parse_output(response)
            # print(response)
            # print("ans:", intern_ans)
            break
        except Exception as e:
            print(e)
            retries -= 1
            time.sleep(30)
            if retries == 0:
                intern_ans = "GPT4 could not answer this question."
                print("retries exhausted")
                break
            continue

    answer = data["correct_option_letter"].strip()
    ytrue.append(answer)
    ypred.append(intern_ans)

In [16]:
for i in range(len(ypred)):
    if ypred[i] == "Llama 3.2 could not answer this question.":
        print(i)

In [ ]:
len(ytrue), len(ypred)

### Result Analysis

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

print(accuracy_score(ytrue, ypred))
print(confusion_matrix(ytrue, ypred))
print(classification_report(ytrue, ypred))

print(Counter(ytrue))
print(Counter(ypred))

In [ ]:
import prettytable
from collections import defaultdict

table = prettytable.PrettyTable()
table.field_names = ["Category", "Total", "Wrong", "Accuracy"]

got_wrong_dict = defaultdict(int)

for i in range(len(ypred)):
    if ypred[i] != ytrue[i]:
        got_wrong_dict[eval_dataset[i]["category"]] += 1
    else:
        got_wrong_dict[eval_dataset[i]["category"]] += 0


for k, v in got_wrong_dict.items():
    table.add_row([k, category_count[k], v, 1 - (v / category_count[k])])


# sort by total
table.sortby = "Total"
table.reversesort = True
print(table)

In [ ]:
METRIC_SAVE_DIR = "../../result_jsons/"

eval_dataset_copy = eval_dataset.copy()

print(len(eval_dataset_copy))
for i, data in enumerate(eval_dataset_copy):
    # map letter to option f
    if "BLOCK" in ypred[i]:
        data["vlm_answer"] = "BLOCK"
    else:
        data["vlm_answer"] = ypred[i]

In [21]:
import json

with open(METRIC_SAVE_DIR + "llama3.2_90b_sofloc_0shot.json", "w") as f:
    json.dump(eval_dataset_copy, f, indent=4)